# Marine Biodiversity and Environmental Data Exploration with `mlcroissant`
This notebook provides a guide for loading and exploring a dataset using the `mlcroissant` library.

### Dataset Source
The dataset source is provided via a Croissant schema URL.

In [ ]:
# Ensure `mlcroissant` library is installed
!pip install mlcroissant

## 1. Data Loading
Load metadata and records from the dataset using `mlcroissant`.

In [ ]:
import mlcroissant as mlc
import pandas as pd

# Define the dataset URL
url = 'https://cdn.dev.senscience.cloud/portals/10.82843/pm80-mh77/fair2.json'

# Load the dataset metadata
dataset = mlc.Dataset(url)
metadata = dataset.metadata.to_json()
print(f"{metadata['name']}: {metadata['description']}")

## 2. Data Overview
Review available record sets, fields, and their IDs.

In [ ]:
# List all record sets
record_sets = [record_set['@id'] for record_set in metadata['recordSet']]
print("Available Record Sets:")
for record_set in record_sets:
    print(f"- {record_set}")

## 3. Data Extraction
Load data from a specific record set into a DataFrame for analysis. Use the record set and field `@id`s from the overview.

In [ ]:
# Extract data from each record set
dataframes = {}

for record_set in record_sets:
    print(f"Loading: {record_set}")
    records = list(dataset.records(record_set=record_set))
    dataframes[record_set] = pd.DataFrame(records)

# Display columns for a specific record set
selected_record_set = 'https://senscience.ai/frontiers/borja/WATER.csv'
print(f"Columns in {selected_record_set}:")
print(dataframes[selected_record_set].columns.tolist())
dataframes[selected_record_set].head()

## 4. Exploratory Data Analysis (EDA)
Apply common data processing steps, such as filtering records based on specific criteria, normalizing numeric fields, and categorizing data. This section should include operations like removing outliers, transforming data distributions, or grouping data by key attributes to prepare it for further analysis.

In [ ]:
# Select a numeric field for analysis
numeric_field = 'https://api.dev.senscience.cloud/frontiers/8704cfcf-0667-4eaf-b0b1-679597df683f-16/31c73ca6-cd75-4ed6-8d60-2cadad090287'

threshold = 10
filtered_df = dataframes[selected_record_set][dataframes[selected_record_set][numeric_field] > threshold]
print(f"Filtered records with {numeric_field} > {threshold}:")
print(filtered_df.head())

# Normalize the numeric field
filtered_df[f"{numeric_field}_normalized"] = (filtered_df[numeric_field] - filtered_df[numeric_field].mean()) / filtered_df[numeric_field].std()
print(f"Normalized {numeric_field} for filtered records:")
print(filtered_df[[numeric_field, f"{numeric_field}_normalized"]].head())

# Group data by a field
group_field = 'https://api.dev.senscience.cloud/frontiers/8704cfcf-0667-4eaf-b0b1-679597df683f-16/2a175358-f06d-4155-afb2-5f866414aa2d'
if group_field in dataframes[selected_record_set].columns:
    grouped_df = filtered_df.groupby(group_field).mean()
    print(f"Grouped data by {group_field}:")
    print(grouped_df.head())

## 5. Visualization
Visualize data distributions or relationships between fields in the dataset.

In [ ]:
# Plotting library
import matplotlib.pyplot as plt

# Example plot
plt.figure(figsize=(10, 6))
plt.hist(filtered_df[numeric_field], bins=50, alpha=0.7, label='Raw Data')
plt.hist(filtered_df[f"{numeric_field}_normalized"], bins=50, alpha=0.7, label='Normalized Data')
plt.title('Distribution of Parameter Values')
plt.xlabel('Parameter Value')
plt.ylabel('Frequency')
plt.legend()
plt.show()

## 6. Conclusion
Summarize key findings and observations from the dataset exploration.